<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">ElasticNet Models with Box Score Features</h4>
    <h5 style="font-weight: bold; font-size: 24px;">Hyperparameter Tuning using Expanding Window</h5>
    <p style="font-size: 20px;">NBA API Seasons 2021-22 to 2023-24</p>
</div>

<a name="Models"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

[Inspect Expanding Training Window](#Inspect-Training-Windows)

**[1. Target: Total Points (over / under)](#1.-Target:-Total-Points-(over-/-under))**
  
**[2. Target: Difference in Points (plus / minus)](#2.-Target:-Difference-in-Points-(plus-/-minus))**

**[3. Target: Game Winner (moneyline)](#3.-Target:-Game-Winner-(moneyline))**

# Setup

[Return to top](#Models)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Data

[Return to top](#Models)

Data splits:

- Define NBA Season 2021-22 as the TRAINING set: regular season is 2021-10-19 to 2022-04-10. 
- Define NBA Season 2022-23 as the VALIDATION set: regular season is 2022-10-18 to 2023-04-09.
- Define NBA Season 2023-24 as the TESTING set: regular season is 2023-10-24 to 2024-04-14.

In [2]:
# load, filter (by time) and scale data
pts_scaled_df, pm_scaled_df, res_scaled_df, test_set_obs = utl.load_and_scale_data(
    input_data='../../data/processed/nba_team_matchups_rolling_box_scores_2022_2024_r05.csv',
    seasons_to_keep=['2021-22', '2022-23', '2023-24'], 
    training_season='2021-22',
    feature_prefixes=['ROLL_'],
    scaler_type='minmax', 
    scale_target=False
)

Season 2021-22: 1186 games
Season 2022-23: 1181 games
Season 2023-24: 692 games
Total number of games across sampled seasons: 3059 games


In [3]:
# define number of games in seasons
season_22_ngames = 1186
season_23_ngames = 1181

In [4]:
pts_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,TOTAL_PTS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,0.745,0.522,0.296,0.753,0.758,0.580,0.731,0.805,0.878,0.535,0.571,0.292,0.478,0.612,1.000,1.000,0.600,0.661,0.577,0.586,0.202,0.704,0.526,0.176,1.000,0.336,0.285,0.603,0.250,0.369,0.291,0.500,0.28,0.5,0.391,0.500,185
2021-10-23,0.000,0.000,0.648,0.000,0.076,0.412,0.000,0.466,0.534,0.438,1.000,0.381,0.826,0.000,0.420,0.273,0.657,0.576,0.096,0.017,0.362,0.000,0.421,0.588,0.364,0.294,0.163,0.837,0.312,0.685,0.606,0.083,0.28,0.3,0.348,0.571,198
2021-10-23,0.691,0.652,0.507,0.758,0.455,0.454,0.466,0.593,0.534,0.720,0.286,0.602,0.609,0.561,0.058,0.364,0.257,0.661,0.635,0.586,0.176,0.728,0.263,0.265,0.396,0.672,0.772,0.469,0.125,0.685,0.488,0.708,0.36,0.2,0.174,0.643,239
2021-10-23,0.727,0.826,0.683,0.827,0.530,0.244,0.772,0.297,0.382,0.315,0.571,0.159,0.348,0.918,0.275,0.182,0.029,0.661,0.250,0.069,0.122,0.225,0.368,0.559,0.317,0.588,0.813,0.268,0.000,0.369,0.134,0.208,0.20,0.0,0.348,0.929,232
2021-10-24,0.745,0.783,0.577,0.848,0.833,0.580,0.820,0.254,0.229,0.560,0.357,0.779,0.826,0.765,0.565,0.818,0.543,0.322,1.000,0.897,1.000,0.362,0.842,1.000,0.559,0.504,0.569,0.446,0.625,0.730,0.843,0.833,0.76,0.9,0.478,0.786,204


In [5]:
pm_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,PLUS_MINUS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,0.745,0.522,0.296,0.753,0.758,0.580,0.731,0.805,0.878,0.535,0.571,0.292,0.478,0.612,1.000,1.000,0.600,0.661,0.577,0.586,0.202,0.704,0.526,0.176,1.000,0.336,0.285,0.603,0.250,0.369,0.291,0.500,0.28,0.5,0.391,0.500,7.0
2021-10-23,0.000,0.000,0.648,0.000,0.076,0.412,0.000,0.466,0.534,0.438,1.000,0.381,0.826,0.000,0.420,0.273,0.657,0.576,0.096,0.017,0.362,0.000,0.421,0.588,0.364,0.294,0.163,0.837,0.312,0.685,0.606,0.083,0.28,0.3,0.348,0.571,-8.0
2021-10-23,0.691,0.652,0.507,0.758,0.455,0.454,0.466,0.593,0.534,0.720,0.286,0.602,0.609,0.561,0.058,0.364,0.257,0.661,0.635,0.586,0.176,0.728,0.263,0.265,0.396,0.672,0.772,0.469,0.125,0.685,0.488,0.708,0.36,0.2,0.174,0.643,29.0
2021-10-23,0.727,0.826,0.683,0.827,0.530,0.244,0.772,0.297,0.382,0.315,0.571,0.159,0.348,0.918,0.275,0.182,0.029,0.661,0.250,0.069,0.122,0.225,0.368,0.559,0.317,0.588,0.813,0.268,0.000,0.369,0.134,0.208,0.20,0.0,0.348,0.929,-10.0
2021-10-24,0.745,0.783,0.577,0.848,0.833,0.580,0.820,0.254,0.229,0.560,0.357,0.779,0.826,0.765,0.565,0.818,0.543,0.322,1.000,0.897,1.000,0.362,0.842,1.000,0.559,0.504,0.569,0.446,0.625,0.730,0.843,0.833,0.76,0.9,0.478,0.786,-10.0


In [6]:
res_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,GAME_RESULT
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,0.745,0.522,0.296,0.753,0.758,0.580,0.731,0.805,0.878,0.535,0.571,0.292,0.478,0.612,1.000,1.000,0.600,0.661,0.577,0.586,0.202,0.704,0.526,0.176,1.000,0.336,0.285,0.603,0.250,0.369,0.291,0.500,0.28,0.5,0.391,0.500,1
2021-10-23,0.000,0.000,0.648,0.000,0.076,0.412,0.000,0.466,0.534,0.438,1.000,0.381,0.826,0.000,0.420,0.273,0.657,0.576,0.096,0.017,0.362,0.000,0.421,0.588,0.364,0.294,0.163,0.837,0.312,0.685,0.606,0.083,0.28,0.3,0.348,0.571,0
2021-10-23,0.691,0.652,0.507,0.758,0.455,0.454,0.466,0.593,0.534,0.720,0.286,0.602,0.609,0.561,0.058,0.364,0.257,0.661,0.635,0.586,0.176,0.728,0.263,0.265,0.396,0.672,0.772,0.469,0.125,0.685,0.488,0.708,0.36,0.2,0.174,0.643,1
2021-10-23,0.727,0.826,0.683,0.827,0.530,0.244,0.772,0.297,0.382,0.315,0.571,0.159,0.348,0.918,0.275,0.182,0.029,0.661,0.250,0.069,0.122,0.225,0.368,0.559,0.317,0.588,0.813,0.268,0.000,0.369,0.134,0.208,0.20,0.0,0.348,0.929,0
2021-10-24,0.745,0.783,0.577,0.848,0.833,0.580,0.820,0.254,0.229,0.560,0.357,0.779,0.826,0.765,0.565,0.818,0.543,0.322,1.000,0.897,1.000,0.362,0.842,1.000,0.559,0.504,0.569,0.446,0.625,0.730,0.843,0.833,0.76,0.9,0.478,0.786,0


# Inspect Expanding Training Window

[Return to top](#Models)

In [7]:
# expanding window configuration
initial_train_size = 10  # starting size of the training set
test_size = 1            # leave-one-out (LOO) cross-validation
gap_size=0               # should there be a gap between train and test sets?
expansion_limit=None     # the limit on the test set observations

counter = 0
max_splits_to_show = 15

# show first few splits
for train_indices, test_indices in utl.expanding_window_ts_split(pts_scaled_df, initial_train_size, 
                                                                 test_size=test_size, gap_size=gap_size,
                                                                 expansion_limit=expansion_limit):
    print("TRAIN:", train_indices, "TEST:", test_indices)
    counter += 1
    if counter >= max_splits_to_show:
        break

TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10] TEST: [11]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11] TEST: [12]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12] TEST: [13]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] TEST: [14]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [15]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [16]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] TEST: [17]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17] TEST: [18]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18] TEST: [19]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] TEST: [21]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] TEST: [22]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22] TEST: [23]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9

<a name="1.-Target:-Total-Points-(over-/-under)"></a>
# 1. Target: Total Points (over / under)

[Return to top](#Models)

In [8]:
# configuration for expanding window
results = utl.train_models_over_grid(
    model_class=ElasticNet, # model class
    target_col='TOTAL_PTS', # target column name
    df=pts_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'fit_intercept': True,
        'max_iter': 10000,
        'selection': 'cyclic'
    },
    explore_params={
        'alpha': [0.001, 0.01, 0.1, 1],   # tried: 1, 10, 75, 100, 125
        'l1_ratio': [0.01, 0.5, 1],   # tried: 0, 0.5, 1
    }
)

Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 0.01}
Total time taken: 0.03 seconds
Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 0.5}
Total time taken: 0.04 seconds
Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 1}
Total time taken: 0.32 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 0.01}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 0.5}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 1}
Total time taken: 0.03 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 0.01}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 0.5}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 1}
Total time taken: 0.02 seconds
Parameters currently explored: {'alpha': 1, 'l1_ratio': 0.01}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 1, 'l1_ratio':

In [9]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,fit_intercept,l1_ratio,max_iter,null_rmse,random_state,selection
2,run_2,0.001,19.203,True,1.00,10000,19.853,599,cyclic
1,run_1,0.001,19.218,True,0.50,10000,19.853,599,cyclic
0,run_0,0.001,19.235,True,0.01,10000,19.853,599,cyclic
5,run_5,0.010,19.249,True,1.00,10000,19.853,599,cyclic
4,run_4,0.010,19.413,True,0.50,10000,19.853,599,cyclic


In [10]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/ELN_pts_best_params_boxscores.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="2.-Target:-Difference-in-Points-(plus-/-minus)"></a>
# 2. Target: Difference in Points (plus / minus)

[Return to top](#Models)

In [11]:
# configuration for expanding window
results = utl.train_models_over_grid(
    model_class=ElasticNet, # model class
    target_col='PLUS_MINUS', # target column name
    df=pm_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'fit_intercept': True,
        'max_iter': 10000,
        'selection': 'cyclic'
    },
    explore_params={
        'alpha': [0.001, 0.01, 0.1, 1, 5],  # tried: 1, 10, 75, 100, 125, 150
        'l1_ratio': [0.01, 0.5, 1]    # tried: 0, 0.5, 1
    }
)

Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 0.01}
Total time taken: 0.04 seconds
Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 0.5}
Total time taken: 0.03 seconds
Parameters currently explored: {'alpha': 0.001, 'l1_ratio': 1}
Total time taken: 0.11 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 0.01}
Total time taken: 0.02 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 0.5}
Total time taken: 0.02 seconds
Parameters currently explored: {'alpha': 0.01, 'l1_ratio': 1}
Total time taken: 0.02 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 0.01}
Total time taken: 0.02 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 0.5}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 0.1, 'l1_ratio': 1}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 1, 'l1_ratio': 0.01}
Total time taken: 0.01 seconds
Parameters currently explored: {'alpha': 1, 'l1_ratio':

In [12]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,fit_intercept,l1_ratio,max_iter,null_rmse,random_state,selection
0,run_0,0.001,13.914,True,0.01,10000,14.253,599,cyclic
1,run_1,0.001,13.918,True,0.50,10000,14.253,599,cyclic
5,run_5,0.010,13.919,True,1.00,10000,14.253,599,cyclic
4,run_4,0.010,13.927,True,0.50,10000,14.253,599,cyclic
2,run_2,0.001,13.928,True,1.00,10000,14.253,599,cyclic


In [13]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/ELN_pm_best_params_boxscores.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="3.-Target:-Game-Winner-(moneyline)"></a>
# 3. Target: Game Winner (moneyline)

[Return to top](#Models)

In [14]:
# configuration for expanding window
results = utl.train_models_over_grid(
    model_class=LogisticRegression, # model class
    target_col='GAME_RESULT', # target column name
    df=res_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=season_23_ngames,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=None, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'n_jobs': -1,
        'solver': 'saga',
        'penalty': 'elasticnet'
    },
    explore_params={
        'C': [1e-50, 1e-35, 1e-25],  # tried: 1e-8, 1e-7, 1e-6, 1e-5, 1e-4
        'l1_ratio': [0.01, 0.5, 1]         # tried: 0, 0.5, 1
    }
)

Parameters currently explored: {'C': 1e-50, 'l1_ratio': 0.01}
Total time taken: 0.06 seconds
Parameters currently explored: {'C': 1e-50, 'l1_ratio': 0.5}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-50, 'l1_ratio': 1}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-35, 'l1_ratio': 0.01}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-35, 'l1_ratio': 0.5}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-35, 'l1_ratio': 1}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-25, 'l1_ratio': 0.01}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-25, 'l1_ratio': 0.5}
Total time taken: 0.03 seconds
Parameters currently explored: {'C': 1e-25, 'l1_ratio': 1}
Total time taken: 0.03 seconds
Total time taken: 0.32 seconds


In [15]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_accuracy', ascending=False).head()

,run_id,C,average_accuracy,average_f1_score,baseline_accuracy,l1_ratio,n_jobs,overall_auc,penalty,pred_labels,random_state,solver
2,run_2,1.000e-50,0.573,0.728,0.573,1.00,-1,0.49,elasticnet,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",599,saga
5,run_5,1.000e-35,0.573,0.728,0.573,1.00,-1,0.49,elasticnet,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",599,saga
8,run_8,1.000e-25,0.573,0.728,0.573,1.00,-1,0.49,elasticnet,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",599,saga
0,run_0,1.000e-50,0.427,0.000,0.573,0.01,-1,0.50,elasticnet,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",599,saga
1,run_1,1.000e-50,0.427,0.000,0.573,0.50,-1,0.50,elasticnet,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",599,saga


In [16]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_accuracy')

# save the dictionary to a file
with open('../../hyperparameters/ELN_res_best_params_boxscores.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)